In [1]:
import concurrent
import os
import re
import shutil
import subprocess
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
from io import StringIO
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
import parmed as pmd
import pytraj as pt

In [7]:
from Plain_MD import Molecular_Dynamics, Run_MD
from Calc_MMPBSA import Wrapper_MMPBSA

In [8]:
# # Run settings
# delta_pico = 0.002
# 
# system_settings = {
#     "receptor_path": "./LAC3.pdb",
#     "ligand_path": "./C4_prep.sdf",
#     "delta_pico": delta_pico,
#     "rerun": False,
#     "gpu_id": "0"
# }
# 
# nvt_settings = {
#     "steps": int(50 // delta_pico),      
#     "dcd_save":int(50 // delta_pico),
#     "log_save":int(1 // delta_pico),
#     "temps_list_simulating":[50, 100, 150, 200, 250, 300, 301]
# }
# 
# npt_settings = {
#     "steps": int(50 // delta_pico),      
#     "dcd_save": int(50 // delta_pico),
#     "log_save": int(1 // delta_pico),
#     "rests_list_decreasing":[1000000000, 100000, 1000, 100, 10, 1],
#     "atoms_to_restraints":{"CA"}
# }
# 
# md_settings = {
#     "steps": int(1000 // delta_pico),     
#     "dcd_save":int(10 // delta_pico),
#     "log_save":int(5 // delta_pico)
# }

In [9]:
# runner = Run_MD()
# runner.equilibration_production(system_settings, nvt_settings, npt_settings, md_settings)

In [10]:
traj_top_paths = {}
for root, dirs, files in os.walk("."):
    for dir in dirs:
        if dir.startswith("C4"):
            dirpath = Path(root) / dir

            # Startup the dictionary
            mydict = {str(dirpath): {"PRMTOP": "", "DCD": ""}}

            for file in dirpath.iterdir():
                if str(file).endswith("m.prmtop"):
                    mydict[str(dirpath)]["PRMTOP"] = str(file.absolute())
                
                if str(file).endswith("0.dcd"):
                    mydict[str(dirpath)]["DCD"] = str(file.absolute())
            
            traj_top_paths.update(mydict)

In [11]:
traj_top_paths

{'C4_prep': {'PRMTOP': '/mnt/c/Users/ricca/Desktop/Documents/1_Coding/1_Script_Github/Chem_Script_Repo/OpenMM_VanillaMD_MMPBSA/examples/C4_prep/system.prmtop',
  'DCD': '/mnt/c/Users/ricca/Desktop/Documents/1_Coding/1_Script_Github/Chem_Script_Repo/OpenMM_VanillaMD_MMPBSA/examples/C4_prep/Step3_Md_Rep0.dcd'}}

In [13]:
mmpbsa_df = Wrapper_MMPBSA(traj_top_paths)()

Stripping :HOH,NA,CL (solvent) from original topology, output is C4_prep/MMPBSA/complex.prmtop
Done stripping solvent!

Creating receptor topology file by stripping !(:UNK) from C4_prep/MMPBSA/complex.prmtop
Done creating receptor topology file!

Creating ligand topology file by stripping :UNK from C4_prep/MMPBSA/complex.prmtop
Done creating ligand topology file!
Loading and checking parameter files for compatibility...
cpptraj found! Using /home/hitesit/mambaforge/envs/cheminf_3_11/bin/cpptraj
mmpbsa_py_energy found! Using /home/hitesit/mambaforge/envs/cheminf_3_11/bin/mmpbsa_py_energy
Preparing trajectories for simulation...
4 frames were processed by cpptraj for use in calculation.

Running calculations on normal system...

Beginning GB calculations with /home/hitesit/mambaforge/envs/cheminf_3_11/bin/mmpbsa_py_energy
  calculating complex contribution...
  calculating receptor contribution...
  calculating ligand contribution...

Beginning PB calculations with /home/hitesit/mambafor

In [14]:
mmpbsa_df

,Ligand,GB,PB
0,C4_prep,-30.418668,3.433125
